In [2]:
#libraries
import torch
from torchvision import models, transforms
from PIL import Image
import pandas as pd
import os
from tqdm import tqdm

#useing gpu 
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
image_folder = "test" 

#giving class names to the model
class_names = ['baked_potato','baklava','caesar_salad','cheese_sandwich','cheesecake',
               'chicken','chicken_curry','chocolate_cake','donuts','eggs','falafel','fish',
               'french_fries','hamburger','hot_dog','ice_cream','lasagna','omelette',
               'pizza','spaghetti','steak','sushi']

#loading the model
model = models.resnet50(weights=None)
model.fc = torch.nn.Linear(model.fc.in_features, len(class_names))
model.load_state_dict(torch.load("finetuned_resnet50_state.pth", map_location=device))
model.to(device)
model.eval()

#preparation
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(
        mean=[0.485, 0.456, 0.406],
        std=[0.229, 0.224, 0.225]
    )
])

#Loading test files
image_files = [f for f in os.listdir(image_folder)]

results = []

#prediction
with torch.no_grad():
    for filename in tqdm(image_files, desc="analyzing"):
        path = os.path.join(image_folder, filename)
        try:
            image = Image.open(path).convert('RGB')
            img_tensor = transform(image).unsqueeze(0).to(device)

            output = model(img_tensor)
            pred = output.argmax(1).item()
            predicted_class = class_names[pred]

            results.append({
                "name": filename,
                "predicted": predicted_class
            })
        except Exception as e:
            print(f"error at {filename}: {e}")


df = pd.DataFrame(results)
df.to_csv("q1_submission.csv", index=False)

print("'q1_submission.csv' done")


analyzing: 100%|██████████| 2200/2200 [00:24<00:00, 89.27it/s]

'q1_submission.csv' done
